<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">OC SWITRS GIS Data Processing</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Part 4 - Sharing ArcGIS Online</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.2.1, May 2025</div>
</center></div>

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">1. Preliminaries</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.1 Referencing Libraries and Initialization</h2>

If needed to reset the kernel, please run the following cell:

In [ ]:
#%reset

Instantiating python libraries for the project

In [1]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from arcpy import metadata as md
from dotenv import load_dotenv

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.2. Project and Workspace Variables</h2>

Define and maintain project, workspace, ArcGIS, and data-related variables

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project and Geodatabase Paths</h3>

Define the ArcGIS pro project variables

In [2]:
# Current notebook directory
notebookDir = os.getcwd()

# Define the project folder (parent directory of the current working directory)
projectFolder = os.path.dirname(os.getcwd())

Which running environment this notebook is using? (1 = Visual Studio Code, 2 = ArcGIS Pro)

In [3]:
runEnv = input("Enter the running environment (1=VSCode, 2=ArcGIS Pro): ")

Load environment variables from the .env file

In [4]:
# Load environment variables from .env file
load_dotenv()

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">ArcGIS Pro Paths</h3>

ArcGIS pro related paths

In [5]:
# OCSWITRS project AGP path
agpFolder = os.path.join(projectFolder, "AGPSWITRS")

# AGP APRX file name and path
aprxName = "AGPSWITRS.aprx"
aprxPath = os.path.join(agpFolder, aprxName)

# ArcGIS Pro project geodatabase and path
gdbName = "AGPSWITRS.gdb"
gdbPath = os.path.join(agpFolder, gdbName)

# ArcGIS pro project
if runEnv == "1":  # VSCode
    print("Running in VSCode (project = aprxPath)")
    aprxObj = aprxPath
elif runEnv == "2":  # ArcGIS Pro
    print("Running in ArcGIS Pro (project = CURRENT)")
    aprxObj = "CURRENT"

try:
    # Set the ArcGIS Pro project based on the environment
    # Use arcpy.mp.ArcGISProject() function to initialize the project
    aprx = arcpy.mp.ArcGISProject(aprxObj)
    
    # Close all map views
    aprx.closeViews()
except Exception as e:
    print(f"Error loading ArcGIS Project: {e}")

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace
# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True
# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

Running in VSCode (project = aprxPath)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Folder Paths</h3>

In [6]:
# Raw data folder path
rawDataFolder = os.path.join(projectFolder, "data", "raw")

# Maps folder path
mapsFolder = os.path.join(projectFolder, "maps")

# Layers folder path
layersFolder = os.path.join(projectFolder, "layers")
layersTemplate = os.path.join(layersFolder, "templates")

# Layouts folder path
layoutsFolder = os.path.join(projectFolder, "layouts")

# Notebooks folder path
notebooksFolder = os.path.join(projectFolder, "notebooks")
codebookPath = os.path.join(projectFolder, "scripts", "codebook", "cb.json")

Geodatabase feature datasets paths (directories)

In [7]:
# RawData feature dataset in the geodatabase
gdbRawData = os.path.join(gdbPath, "raw")

# RawData feature dataset in the geodatabase
gdbSupportingData = os.path.join(gdbPath, "supporting")

# AnalysisData feature dataset in the geodatabase
gdbAnalysisData = os.path.join(gdbPath, "analysis")

# HotSpotData feature dataset in the geodatabase
gdbHotspotData = os.path.join(gdbPath, "hotspots")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Data Folder Paths</h3>

The most current raw data files cover the periods from 01/01/2013 to 09/30/2024. The data files are already processed in the R scripts and imported into the project's geodatabase.

In [8]:
# Add the start date of the raw data to a new python datetime object
dateStart = datetime(2012, 1, 1)

# Add the end date of the raw data to a new python datetime object
dateEnd = datetime(2024, 12, 31)

# Define time and date variables
timeZone = pytz.timezone("US/Pacific")
today = datetime.now(timeZone)
dateUpdated = today.strftime("%B %d, %Y")
timeUpdated = today.strftime("%I:%M %p")

Define date strings for metadata

In [9]:
# String defining the years of the raw data
mdYears = f"{dateStart.year}-{dateEnd.year}"

# String defining the start and end dates of the raw data
mdDates = f"Data from {dateStart.strftime('%B %d, %Y')} to {dateEnd.strftime('%B %d, %Y')}"

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Codebook</h3>

In [10]:
# Load the JSON file from directory and store it in a variable
with open(codebookPath) as jsonFile:
    codebook = json.load(jsonFile)

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.3. ArcGIS Pro Workspace</h2>

Set the workspace and environment settings for the ArcGIS Pro project

In [11]:
# Set the workspace and environment to the root of the project geodatabase
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True

# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.4. Map and Layout Lists</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Maps</h3>

In [12]:
# List of maps to be created for the project
mapList = ["collisions", "crashes", "parties", "victims", "injuries", "fatalities", "fhs100m1km", "fhs150m2km", "fhs100m5km", "fhsRoads500ft", "ohsRoads500ft", "roadCrashes",
           "roadHotspots", "roadBuffers", "roadSegments", "roads", "pointFhs", "pointOhs", "popDens", "houDens", "areaCities", "areaBlocks", "summaries", "analysis", "regression"]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Layouts</h3>

In [13]:
# List or layouts to be created for the project
layoutList = ["maps", "injuries", "hotspots", "roads", "points", "densities", "areas"]

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">3. ArcGIS Online Operations</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.1. Sign In</h2>

In [14]:
# Sign in to portal
arcpy.SignInToPortal("https://www.arcgis.com", os.getenv("AGO_USERNAME"), os.getenv("AGO_PASSWORD"))

{'token': 'mzFcMRqhxzPAoRJavp2MJn0VBFp6Imvpp-5I8rbezHt22Iy2IyTCurpOxzXl3rUU9PYgvHfUuyXeHnaEE2Fb5sFK-QyVfdC26GuDDNiiq1owC3ZbiQl7u0h-RoRF8lWAcVMfXxTkN6LcYImdYfOsZRyQTVVMJcKhv5IaYfi9W4jyTEWWK9NbG9tk3HcA4Dw90WkqfOIyKo0irTHIa_L-RAr2xwoqTWHVLajCDc44OGI.',
 'referer': 'http://www.esri.com/AGO/0C650116-F38F-4200-9D5F-57B07485ED38',
 'expires': 1746824839}

In [15]:
# Get the portal description and key information
portalDesc = arcpy.GetPortalDescription()

# Print Basic Portal Information
print(f"Portal: {portalDesc['name']}")
print(f"User: {portalDesc['user']['fullName']} ({portalDesc['user']['username']})")

Portal: OC Public Works
User: Kostas Alexandridis (ktalexan)


Path to store ArcGIS Online staging feature layers

In [37]:
agoPrjPath = os.path.join(projectFolder, "data", "ago")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.2. Maps and Feature Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Collisions Map and Supporting Layers</h3>

Maps and Layers Definitions for the Collisions Map

In [19]:
collisionsMap = aprx.listMaps("collisions")[0]
collisionsLayer = collisionsMap.listLayers("OCSWITRS Collisions")[0]
roadsLayer = collisionsMap.listLayers("OCSWITRS Roads")[0]
blocksLayer = collisionsMap.listLayers("OCSWITRS Census Blocks")[0]
citiesLayer = collisionsMap.listLayers("OCSWITRS Cities")[0]
boundariesLayer = collisionsMap.listLayers("OCSWITRS Boundaries")[0]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes Map and Layer</h3>

Crashes Map and layer definitions

In [34]:
crashesMap = aprx.listMaps("crashes")[0]
crashesLayer = crashesMap.listLayers("OCSWITRS Crashes")[0]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Parties Map and Layer</h3>

Parties map and layer definitions

In [35]:
partiesMap = aprx.listMaps("parties")[0]
partiesLayer = partiesMap.listLayers("OCSWITRS Parties")[0]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Victims Map and Layer</h3>

Victims map and layer definitions

In [36]:
victimsMap = aprx.listMaps("victims")[0]
victimsLayer = victimsMap.listLayers("OCSWITRS Victims")[0]

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.2. Sharing Supporting Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Boundaries Service Layer</h3>

Service definition variables

In [41]:
# Provide the service definition variables for the Boundaries layer
serviceName = "OC Boundaries"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = collisionsMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, boundariesLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the boundaries layer

In [42]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OC Boundaries.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [43]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Cities Service Layer</h3>

Service definition variables

In [45]:
# Provide the service definition variables for the Cities layer
serviceName = "OCSWITRS Cities"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = collisionsMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, citiesLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the boundaries layer

In [46]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Cities.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [47]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Roads Service Layer</h3>

Service definition variables

In [48]:
# Provide the service definition variables for the Roads layer
serviceName = "OCSWITRS Roads"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = collisionsMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, roadsLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the roads layer

In [49]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Roads.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [51]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Census Blocks Service Layer</h3>

Service definition variables

In [53]:
# Provide the service definition variables for the Blocks layer
serviceName = "OCSWITRS Census Blocks"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = collisionsMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, blocksLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the blocks layer

In [54]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Census Blocks.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [55]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. Sharing SWITRS Data Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes Service Layer</h3>

Service definition variables

In [56]:
# Provide the service definition variables for the crashes layer
serviceName = "OCSWITRS Crashes"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = crashesMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, crashesLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the crashes layer

In [57]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Crashes.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [58]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Parties Service Layer</h3>

Service definition variables

In [59]:
# Provide the service definition variables for the parties layer
serviceName = "OCSWITRS Parties"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = partiesMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, partiesLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the parties layer

In [60]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Parties.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [61]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Victims Service Layer</h3>

Service definition variables

In [62]:
# Provide the service definitions variables for the victims layer
serviceName = "OCSWITRS Victims"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = victimsMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, victimsLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the victims layer

In [63]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Victims.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [64]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Collisions Service Layer</h3>

Service definition variables

In [65]:
# Provide the service definitions variables for the collisions layer
serviceName = "OCSWITRS Collisions"
serviceFolder = "OCTraffic"
serviceCredits = "Dr. Kostas Alexandridis, GISP"
sddraftFile = serviceName + ".sddraft"
sddraftOutput = os.path.join(agoPrjPath, sddraftFile)
sdFile = serviceName + ".sd"
sdOutput = os.path.join(agoPrjPath, sdFile)

# Define the server type
serverType = "HOSTING_SERVER"

# Create a feature service draft object
sddraft = collisionsMap.getWebLayerSharingDraft(serverType, "FEATURE", serviceName, collisionsLayer)

# Set the properties of the feature service draft object
sddraft.overwriteExistingService = True
sddraft.portalFolder = serviceFolder
sddraft.credits = serviceCredits

# Create Service Definition Draft file
sddraft.exportToSDDraft(sddraftOutput)

Stage the service definition for the collisions layer

In [66]:
print("Start Staging")
arcpy.server.StageService(
    in_service_definition_draft = sddraftOutput,
    out_service_definition = sdOutput
)

Start Staging


<Result 'c:\\Users\\ktale\\OneDrive\\Documents\\GitHub\\OCSWITRS\\data\\ago\\OCSWITRS Collisions.sd'>

Publish the service definition to ArcGIS Online portal (updating the existing service)

In [67]:
# Share to portal
print("Start Uploading")
arcpy.server.UploadServiceDefinition(
    in_sd_file = sdOutput,
    in_server = serverType,
    in_service_name = serviceName,
    in_folder_type = "EXISTING",
    in_folder = serviceFolder,
    in_startupType = "STARTED"
)

Start Uploading


<Result ''>